In [84]:
#importing modules for the implementation
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
import lightgbm as lgb
from sklearn import metrics
import seaborn as sns
import xgboost as xgb



In [69]:
import re
nutrition_test = pd.read_csv("Test_20.csv")
nutrition_train = pd.read_csv("Train_80.csv")
nutrition_test = nutrition_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
nutrition_train = nutrition_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
nutrition_test = nutrition_test.dropna()
nutrition_train = nutrition_train.dropna()

In [70]:
#extracting the nutrition data, labelling X as this is the input to the SKlearn algorithm
X_train = nutrition_train.iloc[:,2:42]
nutrition_titles = X_train.columns
X_test = nutrition_test.iloc[:,2:42]
#extracting the greenhouse gas emissions 
y_test = nutrition_test.iloc[:,43:44]
y_train = nutrition_train.iloc[:,43:44]
y_test = np.ravel(y_test)
y_train = np.ravel(y_train)

In [71]:
model = lgb.LGBMRegressor()
model.fit(X_train, y_train)
print(); print(model)

expected_y  = y_test
predicted_y = model.predict(X_test)


LGBMRegressor()


In [72]:
print(metrics.r2_score(expected_y, predicted_y))
print(metrics.mean_absolute_error(expected_y, predicted_y))

0.8840492626454673
0.25103951919082945


## Hyperparameter tuning

In [97]:
params = {
    # Parameters that i am going to tune.
    'max_depth':6,
    'num_leaf': 4,
    'learning_rate':.05,
    'max_bin': 255,
    'min_data_in_leaf': 4,
    # Other parameters
    'objective':'regression',
}

## Numboost and early stopping rounds for test set

In [111]:
from datetime import datetime
train_data=lgb.Dataset(X_train,label=y_train)
test_data=lgb.Dataset(X_test,label=y_test)
# evaluation metric
#params['eval_metric'] = "mae"
# num boosting rounds is set to a high number as we want to allow time 
# for the optimal boosting round to be found. 
num_boost_round = 999
#training our model using light gbm
num_round=50
lgbm=lgb.train(params,train_data,num_round)


[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000993 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6839
[LightGBM] [Info] Number of data points in the train set: 832, number of used features: 40
[LightGBM] [Info] Start training from score 0.868784


In [105]:
#training our model using light gbm
num_round=50
start=datetime.now()
lgbm=lgb.train(params,train_data,num_round)
stop=datetime.now()

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001293 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6839
[LightGBM] [Info] Number of data points in the train set: 832, number of used features: 40
[LightGBM] [Info] Start training from score 0.868784


In [106]:
#Execution time of the model
execution_time_lgbm = stop-start
execution_time_lgbm

datetime.timedelta(microseconds=36895)

In [110]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


#predicting on test set
pred=lgbm.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))


Testing performance
RMSE: 0.63
R2: 0.79


### Attempted to set up a random hyperparameter tuner, unsuccessful, i think because of the size of the data set, the model is overfitting really quickly, or that the num of leaves is too big. 

In [78]:
#Set the minimum error arbitrarily large
min = 99999999999999999999999 
count = 0 #Used for keeping track of the iteration number
#How many runs to perform using randomly selected hyperparameters
iterations = 20
for i in range(iterations):
    print('iteration number', count)
    count += 1 #increment count
    try:
        d_train = lgb.Dataset(X_train, label=y_train) #Load in data
        params = {} #initialize parameters
        params['learning_rate'] = np.random.uniform(0, 1)
        params['boosting_type'] = np.random.choice(['gbdt', 'dart', 'goss'])
        params['objective'] = 'regression'
        params['metric'] = 'mae'
        params['sub_feature'] = np.random.uniform(0, 1)
        params['num_leaves'] = np.random.randint(1, 15)
        params['min_data'] = np.random.randint(1, 30)
        params['max_depth'] = np.random.randint(5, 200)
        iterations = np.random.randint(10, 10000)
        print(params, iterations)
        #Train using selected parameters
        clf = lgb.train(params, d_train, iterations)
        y_pred=clf.predict(X_test) #Create predictions on test set
        mae=mean_absolute_error(y_pred,y_test)
        print('MAE:', mae)
        if mae < min:
            min = mae
            pp = params 
    except: #in case something goes wrong
        print('failed with')
        print(params)
    print("*" * 50)
    print('Minimum is: ', min)

iteration number 0
{'learning_rate': 0.5977991198051446, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.7801180115006612, 'num_leaves': 2, 'min_data': 8, 'max_depth': 149} 9862
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001215 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6839
[LightGBM] [Info] Number of data points in the train set: 832, number of used features: 40
[LightGBM] [Info] Start training from score 0.868784
failed with
{'learning_rate': 0.5977991198051446, 'boosting_type': 'dart', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.7801180115006612, 'num_leaves': 2, 'min_data': 8, 'max_depth': 149}
**************************************************
Minimum is:  99999999999999999999999
iteration number 1
{'learning_rate': 0.721013387753628, 'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.1360801

failed with
{'learning_rate': 0.13752484624885397, 'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.7656988127378453, 'num_leaves': 5, 'min_data': 14, 'max_depth': 148}
**************************************************
Minimum is:  99999999999999999999999
iteration number 11
{'learning_rate': 0.324489888615771, 'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.08144874127767421, 'num_leaves': 3, 'min_data': 21, 'max_depth': 54} 3386
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000272 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6839
[LightGBM] [Info] Number of data points in the train set: 832, number of used features: 40
[LightGBM] [Info] Start training from score 0.868784
failed with
{'learning_rate': 0.324489888615771, 'boosting_type': 'gbdt', 'objecti

failed with
{'learning_rate': 0.1726001738082017, 'boosting_type': 'gbdt', 'objective': 'regression', 'metric': 'mae', 'sub_feature': 0.41257429077253116, 'num_leaves': 5, 'min_data': 17, 'max_depth': 8}
**************************************************
Minimum is:  99999999999999999999999
